In [6]:
#import the necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import tweepy
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Aniket
[nltk_data]     Kambli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import json
import csv
import re

In [4]:

def search_for_hashtags(hashtag_phrase):
    CK="uDGavGkVAeqH0UvDIdgGnils4"
    CS="6pKYE0lBshuPTXdRMDAxILXMhJ2e1xL1zM4V85IYNM4aQDoA5T"
    AT="935371370128535552-wOckNr2QW80FsOi4W9WgG643KyYFQuC"
    ATS="gixV61kGNTOGNq2jutfoVOgrcVrG9MMg0HoswJhEIweuy"
    # Authenticate to Twitter
    auth = tweepy.OAuthHandler(CK,CS)
    auth.set_access_token(AT,ATS)

    # Create API object
    api = tweepy.API(auth, wait_on_rate_limit=True,
        wait_on_rate_limit_notify=True)
        #get the name of the spreadsheet we will write to
    #fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))
    fname = hashtag_phrase

        #open the spreadsheet we will write to
    with open('%s.csv' % (fname), 'w',encoding='utf-8') as file:

        w = csv.writer(file)

            #write header row to spreadsheet
        w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags'])

            #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', \
                                       lang="en", tweet_mode='extended').items():
                w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.screen_name.encode('utf-8')])

In [7]:
hashtag_phrase =input('Hashtag Phrase ')

if __name__ == '__main__':
    search_for_hashtags(hashtag_phrase)

Hashtag Phrase Jio


KeyboardInterrupt: 

In [8]:
data=pd.read_csv(hashtag_phrase+'.csv')

In [9]:
data.head()

,timestamp,tweet_text,username,all_hashtags
0,2019-12-11 06:42:26,"b""@JioCare I'm already having a Jio sim but pl...",b'sumeet_divatagi',NaN
1,2019-12-11 06:41:49,"b""How does an Amendment Bill become an act? ...",b'OfficialJioTV',NaN
2,2019-12-11 06:41:47,b'@reliancejio dear sir good morning my name i...,b'Nikhilgoutam92',NaN
3,2019-12-11 06:40:49,b'@sumeet_divatagi We would be glad to welcome...,b'JioCare',NaN
4,2019-12-11 06:39:59,"b""@reliancejio I'm also jio cinema users Sout...",b'Maneesh32095718',NaN


In [10]:
data['tweet_text']=data['tweet_text'].str.replace('\W',' ')

In [13]:
data['tweet_text_cleaned']=data['tweet_text'].apply(remove_b)

In [12]:
#a function to remove 'b' that is occuring in the tweets
def remove_b(s):
    return (s.split(' ',1)[1])

In [14]:
#removing digits from the data
data['tweet_text_cleaned']=data['tweet_text_cleaned'].str.replace('\d+', ' ')

In [15]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

In [16]:
data['tweet_text_cleaned']=data['tweet_text_cleaned'].apply(lemmatizer.lemmatize)

In [18]:
data['tweet_text_cleaned'].head()

0     JioCare I m already having a Jio sim but plan...
1    How does an Amendment Bill become an act    He...
2     reliancejio dear sir good morning my name is ...
3     sumeet_divatagi We would be glad to welcome y...
4     reliancejio I m also jio cinema users  South ...
Name: tweet_text_cleaned, dtype: object

In [17]:
#Using a pretrained model to identify the sentiments
import pickle
filename = 'finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
loaded_vec = pickle.load(open("vector.pickel", "rb" ) )

In [20]:
test=loaded_vec.transform(data['tweet_text_cleaned'])

In [21]:
ypreds=loaded_model.predict(test)

In [22]:
ypreds=list(ypreds)

In [23]:
#making two variables to store the count of positive and negative tweets predicted by the model
positives=0
negatives=0
for x in ypreds:
    if(x==1):
        positives=positives+1
    else:
        negatives=negatives+1

In [24]:
print('positive percentage for',hashtag_phrase,'is:',positives/len(ypreds)*100)

positive percentage for Jio is: 68.44444444444444


In [25]:
print('negative percentage for',hashtag_phrase,'is:',negatives/len(ypreds)*100)

negative percentage for Jio is: 31.555555555555554


In [26]:
#So far we have used our own model, now we will perform sentiment analysis using vader sentiment analyzer

In [27]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [28]:
sentimentanalyzer=SentimentIntensityAnalyzer()

In [31]:
data['vader analyzer']=data['tweet_text_cleaned']

In [35]:
def vader_analyze(s):
    score=sentimentanalyzer.polarity_scores(s)
    if score['compound'] >= 0.05 : 
        return 1 
  
    elif score['compound'] <= - 0.05 : 
        return -1 
  
    else : 
        return 0

In [37]:
data['vader analyzer']=data['vader analyzer'].apply(vader_analyze)

In [38]:
ypreds=list(data['vader analyzer'])

In [40]:
#making two variables to store the count of positive and negative and neutral tweets predicted by the vader model
positives=0
negatives=0
neutral=0
for x in ypreds:
    if(x==1):
        positives=positives+1
    if(x==-1):
        negatives=negatives+1
    else:
        neutral=neutral+1

In [41]:
print('positive percentage for',hashtag_phrase,' using vader is:',positives/len(ypreds)*100)
print('negative percentage for',hashtag_phrase,' using vader is:',negatives/len(ypreds)*100)
print('neutral percentage for',hashtag_phrase,' using vader is:',neutral/len(ypreds)*100)

positive percentage for Jio  using vader is: 59.55555555555555
negative percentage for Jio  using vader is: 18.22222222222222
neutral percentage for Jio  using vader is: 81.77777777777779
